In [463]:
import os
from packaging import version
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb
%matplotlib notebook
plt.rcParams.update(plt.rcParamsDefault)
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Helvetica"
# })
import warnings
warnings.filterwarnings('ignore')


In [464]:
# experiment_id, experiment_name = "6yl48QRjSySDUpTg8TpeRw", "st_uc-bg_bigBugfixMetric_bs4_split_1_1_trial1_29d20f8"
# experiment_id, experiment_name = "zNo4qyA5RBybC9m0sPRiZw", "st_uc-bg_bigBugfixMetric_bs4_split_1_1_trial1_b4f39ae"
# experiment_id, experiment_name = "7M4Qi6dmQteoRpuOibFbWQ", "st_fg_uc_bg_pl_o_95_85_85_s_0_26729b3"

# experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
# df = experiment.get_scalars()
# df.to_csv(f"metrics_{experiment_name}.csv", index=False)

# df = pd.read_csv(f"metrics_{experiment_name}.csv")

In [465]:
def draw_metrics(dfs, fname, tags=None, tag_names=None, threshs=None, y_label=None, colors=None):
    f, axes = plt.subplots(1, 2, sharey=True, figsize=(10,4))
    base_run = "tensorboard"
    if y_label is None:
        y_label = "ROI IoU wrt. pseudo-labels"
    # classes = ["Car", "Pedestrian", "Cyclist"]
    classes = ["Car", "Pedestrian"]
    if tags is None:
        tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_tp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fn"]
    if tag_names is None:
        tag_names = ["True Positive ROIs", "False Positive ROIs", "False Negative ROIs"]
    if colors is None : 
        colors = ['green', 'blue', 'red']
    for j, cls in enumerate(classes):
        for exp in range(2):
            df = dfs[exp]
            for k, (tag, tname, color) in enumerate(zip(tags, tag_names, colors)):
                tname += '/after' if exp == 1 else '/before'
                linestyle = 'solid' if exp == 1 else 'dashed'
                run = base_run + "/" + tag + "/" + cls
                steps = df[df.run == run].step
                values = df[df.run == run].value
                smooth_values = df[df.run == run].ewm(alpha=0.2).mean().value
                if j == 0:
                    lines = axes[j].plot(steps, values, alpha=0.1, color=color)
                    axes[j].plot(steps, smooth_values, label=tname, color=color, linestyle=linestyle)
                    axes[j].set_ylabel(y_label)
                else:
                    lines = axes[j].plot(steps, values, alpha=0.1, color=color)
                    axes[j].plot(steps, smooth_values, color=color, linestyle=linestyle)

            axes[j].yaxis.set_tick_params(labelleft=True)

            if j == 0:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j], linestyle="dashed", label="$\\tau$")
            if j == 1 or j == 2:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j], linestyle="dashed")
                axes[j].yaxis.set_tick_params(labelleft=False)

            axes[j].set_title(cls)
            axes[j].xaxis.set_tick_params(labelbottom=False)
            axes[j].set_xlabel("Training iterations")

    axes[0].legend()
    f.tight_layout()
    f.subplots_adjust(wspace=0.02, hspace=0.02)
    f.savefig(fname=fname, transparent=False, bbox_inches='tight')

In [466]:
def draw_metrics_iou(dfs, fname, tags=None, tag_names=None, threshs=None, y_label=None, colors = None):
    # f, axes = plt.subplots(1, 3, sharey=True, figsize=(13,4))
    f, axes = plt.subplots(2, 3, sharey=True, figsize=(10,5))
    base_run = "tensorboard"
    if y_label is None:
        y_label = "ROI IoU wrt. pseudo-labels"
    # classes = ["Car", "Pedestrian", "Cyclist"]
    classes = ["Car", "Pedestrian"]
    if tags is None:
        tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_tp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fn"]
    if tag_names is None:
        tag_names = ["True Positive ROIs", "False Positive ROIs", "False Negative ROIs"]
    if colors is None : 
        colors = ['green', 'blue', 'red']
    for i in range(2):
        df = dfs[i]
        for j, cls in enumerate(classes):
            for k, (tag, tname, color) in enumerate(zip(tags, tag_names, colors)):
                run = base_run + "/" + tag + "/" + cls
                steps = df[df.run == run].step
                values = df[df.run == run].value
                smooth_values = df[df.run == run].ewm(alpha=0.2).mean().value
                if j == 0:
                    lines = axes[i, j].plot(steps, values, alpha=0.2, color=color)
                    axes[i, j].plot(steps, smooth_values, label=tname, color=color)
                    axes[i, j].set_ylabel(y_label)
                else:
                    lines = axes[i, j].plot(steps, values, alpha=0.2, color=color)
                    axes[i, j].plot(steps, smooth_values, color=color)

            axes[i, j].yaxis.set_tick_params(labelleft=True)

            if j == 0:
                if threshs is not None:
                    axes[i, j].axhline(y=threshs[i, j], linestyle="dashed", label="$\\tau$")
            if j == 1 or j == 2:
                if threshs is not None:
                    axes[i, j].axhline(y=threshs[i, j], linestyle="dashed")
                axes[i, j].yaxis.set_tick_params(labelleft=False)
            if i == 0:
                axes[i, j].set_title(cls)
                axes[i, j].xaxis.set_tick_params(labelbottom=False)
            if i == 1:
                axes[i, j].set_xlabel("Training iterations")

    axes[0, 0].legend()
    f.tight_layout()
    f.subplots_adjust(wspace=0.02, hspace=0.02)
    f.savefig(fname=fname, transparent=False, bbox_inches='tight')

In [467]:
def draw_metrics_correlation(dfs, fname, tags=None, tag_names=None, threshs=None, y_label=None, colors=None):
    f, axes = plt.subplots(2, 1, sharey=True, figsize=(5,5))
    base_run = "tensorboard"
    # classes = ["Car", "Pedestrian", "Cyclist"]
    classes = ["Car", "Pedestrian"]
    x_y_values = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl", "rcnn_roi_pl_gt_metrics_cls/pred_weight"]
    tags = ["tp", "fp", "fn"]
    if colors is None : 
        colors = ['green', 'blue', 'red']
    subplot_titles = ['Before', 'After']
    df = dfs[1]
    for cls_idx, cls in enumerate(classes):
        for tag_idx, (tag, color) in enumerate(zip(tags, colors)):
            iou_tag = base_run + "/" + x_y_values[0] + "_" + tag + "/" + cls
            weight_tag = base_run + "/" + x_y_values[1] + "_" + tag + "/" + cls
            weight = df[df.run == weight_tag].value
            iou = df[df.run == iou_tag].value
            if cls_idx == 0:
                axes[cls_idx].scatter(iou, weight, label=tag, color=color, alpha = 0.5)
            else:
                axes[cls_idx].scatter(iou, weight, color=color, alpha = 0.5)
            axes[cls_idx].yaxis.set_tick_params(labelleft=True)
        
        axes[cls_idx].set_title(cls)
        axes[cls_idx].xaxis.set_tick_params(labelbottom=False)
        axes[cls_idx].set_xlabel("IoUs")
        axes[cls_idx].set_ylabel(y_label)
        axes[cls_idx].legend()

    f.tight_layout()
    f.subplots_adjust(wspace=0.02, hspace=0.2)
    f.savefig(fname=fname, transparent=False, bbox_inches='tight')

In [468]:
def draw_metrics_fg_ious_old(dfs, fname, tags=None, tag_names=None, threshs=None, y_label=None, colors=None):
    f, axes = plt.subplots(2, 1, sharey=True, figsize=(6,6))
    base_run = "tensorboard"
    if y_label is None:
        y_label = "ROI IoU wrt. pseudo-labels"
    classes = ["Car", "Pedestrian", "Cyclist"]
    # classes = ["Car", "Pedestrian"]
    if tags is None:
        tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_tp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fn"]
    if tag_names is None:
        tag_names = ["True Positive ROIs", "False Positive ROIs", "False Negative ROIs"]
    if colors is None : 
        colors = ['green', 'blue', 'red']
    subplot_titles = ['Before', 'After']
    for j in range(len(subplot_titles)):
        df = dfs[j]
        for cls_idx, (cls, color) in enumerate(zip(classes, colors)):
            for k, (tag, tname) in enumerate(zip(tags, tag_names)):
                run = base_run + "/" + tag + "/" + cls
                steps = df[df.run == run].step
                values = df[df.run == run].value
                smooth_values = df[df.run == run].ewm(alpha=0.2).mean().value
                if j == 0:
                    # lines = axes[j].plot(steps, values, alpha=0.1, color=color)
                    axes[j].plot(steps, smooth_values, label=cls, color=color)
                    sns.lineplot(ax=axes[j], data=smooth_values, x=steps, linewidth=2.5)
                else:
                    # lines = axes[j].plot(steps, values, alpha=0.1, color=color)
                    # axes[j].plot(steps, smooth_values, color=color)
                    sns.lineplot(ax=axes[j], data=smooth_values, x=steps, linewidth=2.5)
                axes[j].yaxis.set_tick_params(labelleft=True)

            if j == 0 and cls_idx == 0:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j][cls_idx], linestyle="dashed", label="$\\tau$", color='black')
            if j == 1:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j][cls_idx], linestyle="dashed", label=("$\\tau_{cls}}$".replace('cls', cls)), color=color)
                axes[j].yaxis.set_tick_params(labelleft=True)

        # axes[j].set_title(subplot_titles[j])
        axes[j].xaxis.set_tick_params(labelbottom=False)
        axes[j].set_xlabel("Training iterations")
        axes[j].set_ylabel(y_label)
        axes[j].legend()

    f.tight_layout()
    f.subplots_adjust(wspace=0.02, hspace=0.02)
    f.savefig(fname=fname, transparent=False, bbox_inches='tight')

Draw metrics for FG IoUs between ROIs and PLs

In [499]:
def draw_metrics_fg_ious(dfs, fname, tags=None, tag_names=None, threshs=None, y_label=None, colors=None):
    f, axes = plt.subplots(2, 1, sharey=True, figsize=(6,4))
    base_run = "tensorboard"
    if y_label is None:
        y_label = "ROI IoU wrt. pseudo-labels"
    classes = ["Car", "Pedestrian", "Cyclist"]
    if tags is None:
        tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_tp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fn"]
    if tag_names is None:
        tag_names = ["True Positive ROIs", "False Positive ROIs", "False Negative ROIs"]
    if colors is None : 
        colors = sns.color_palette('tab10', len(classes))
        palette = {mode: color for mode, color in zip(classes, colors)}
    subplot_titles = ['Before', 'After']
    
    for j in range(len(subplot_titles)):
        df = dfs[j]
        classwise_df=pd.DataFrame()
        for cls_idx, (cls, color) in enumerate(zip(classes, colors)):
            for k, (tag, tname) in enumerate(zip(tags, tag_names)):
                run = base_run + "/" + tag + "/" + cls
                steps = df[df.run == run].step
                values = df[df.run == run].value
                smooth_values = df[df.run == run].ewm(alpha=0.2).mean().value
                # store all classwise IoUs in a single dataframe and plot all at once
                classwise_df[cls] = smooth_values.reset_index(drop=True)
            
            # plot threshold lines 
            if j == 0 and cls_idx == 0:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j][cls_idx], linestyle="dashed", label="$\\tau$", color='black', alpha=0.9)
            if j == 1:
                if threshs is not None:
                    axes[j].axhline(y=threshs[j][cls_idx], linestyle="dashed", label=("$\\tau_{cls}$".replace('cls', cls)), color=color, alpha=0.9)
                axes[j].yaxis.set_tick_params(labelleft=True)
        
        # plot ious
        if j == 0:
            sns.lineplot(ax=axes[j], data=classwise_df, linewidth=1.5, dashes=False, legend = False, palette=palette)
        else:
            sns.lineplot(ax=axes[j], data=classwise_df, linewidth=1.5, dashes=False, legend = True, palette=palette)
        
        # x/y ticks
        axes[j].set_yticks(np.arange(0.15, 0.8, 0.15))
        axes[j].yaxis.set_tick_params(labelleft=True, labelsize='small')
        axes[j].xaxis.set_tick_params(labelbottom=False)
        axes[j].yaxis.grid(True)
        # legends
        handles, labels = axes[j].get_legend_handles_labels()
        order = [0,3,1,4,2,5] if j == 1 else [0]

        # Add IoU in legends if needed using below code
        # Note : Needs latex related libraries installed in the env
        # new_labels = []
        # new_handles = []
        # for idx in order:
        #     new_handles.append(handles[idx])
        #     if idx%2!=0:
        #         new_labels.append(r"IoU_{cls}")
        #     else:
        #         new_labels.append(labels[idx])
        # legend = axes[j].legend(new_handles,new_labels, frameon = 1, ncol=3)
        legend = axes[j].legend([handles[idx] for idx in order], [labels[idx] for idx in order], \
                                frameon = 1, ncol=3, fontsize='small')
        frame = legend.get_frame()
        frame.set_color('white')
        frame.set_edgecolor('black')

    # common plot labels
    f.text(0.5, 0.01, "Training iterations", ha='center')
    f.text(0, 0.5, y_label, va='center', rotation='vertical')
    f.tight_layout()
    f.subplots_adjust(wspace=0.02, hspace=0.04)
    f.savefig(fname=fname, transparent=False, bbox_inches='tight')

In [500]:
# Plot setting and theme
plt.style.use('seaborn')
sns.set_theme(style="ticks", palette="tab10", rc={
    'xtick.bottom': False,
    'ytick.left': True,
})

# --------------- Ablation 1 ---------------
exp_id_ab1_default, exp_name_ab1_default = "dTrxKmgMRQK122Hm1cmaPA", "ablation1_bs8_split_1_1_trial_3_5802d99"
exp_id_ab1_ours, exp_name_ab1_ours = "f5nMZ6K9SySkIYEEP1J2jw", "st_fg_uc_bg_pl_o_95_85_85_s_0_26729b3"
df_ab1_default = pd.read_csv(f"metrics_{exp_name_ab1_default}.csv")
df_ab1_ours = pd.read_csv(f"metrics_{exp_name_ab1_ours}.csv")
dfs = [df_ab1_default, df_ab1_ours]

# --------------- Raw IoUs based on FG ---------------
tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fg"]
y_label = "FG IoUs of ROIs and pseudo-labels"
# tag_names = ["FG IoUs of ROIs and pseudo-labels"]
fname = f"fg_roi_iou_wrt_pl_ab1_default_{exp_name_ab1_default}.png"
threshs = np.array([[0.75, 0.75, 0.75], [0.65, 0.45, 0.4]])
draw_metrics_fg_ious(dfs=dfs, fname=fname, threshs=threshs, tags=tags, y_label=y_label)

In [501]:
# # --------------- TP/FP/FN rates ---------------
# tags = ["rcnn_roi_pl_gt_metrics_cls/pred_tp_rate", "rcnn_roi_pl_gt_metrics_cls/pred_fp_ratio", "rcnn_roi_pl_gt_metrics_cls/pred_fn_rate"]
# tag_names = ["TP Rate", "FP Ratio", "FN Rate"]
# y_label = "Ratio"
# colors = ['green', 'blue', 'red']
# draw_metrics(dfs=dfs, fname=f"fp_fn_{exp_name_ab1_default}.png", tags=tags[1:], y_label=y_label, tag_names=tag_names[1:], colors = colors[1:])
# draw_metrics(dfs=dfs, fname=f"tp_{exp_name_ab1_default}.png", tags=[tags[0]], y_label=y_label, tag_names=[tag_names[0]])

# # --------------- TP/FP/FN weights ---------------
# tags = ["rcnn_roi_pl_gt _metrics_cls/pred_weight_tp", "rcnn_roi_pl_gt_metrics_cls/pred_weight_fp", "rcnn_roi_pl_gt_metrics_cls/pred_weight_fn"]
# tag_names = ["TP weights", "FP weights", "FN weights"]
# y_label = "Mean assignment weight"
# draw_metrics(dfs=dfs, fname=f"pred_weights_tp_fp_fn_{exp_name_ab1_default}.png", tags=tags, y_label=y_label, tag_names=tag_names)

# # --------------- Raw IoUs based on TP/FP/FN ---------------
# tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_tp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fp", "rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fn"]
# y_label = "ROI IoU wrt. pseudo-labels"
# fname = f"roi_iou_wrt_pl_ab1_default_{exp_name_ab1_default}.png"
# threshs = np.array([[0.75, 0.75, 0.75], [0.65, 0.45, 0.4]])
# draw_metrics_iou(dfs=dfs, fname=fname, threshs=threshs, tags=tags, y_label=y_label)

# # --------------- Correlation between FG IoUs and weights ---------------
# tags = ["rcnn_roi_pl_gt_metrics_cls/pred_ious_wrt_pl_fg", "rcnn_roi_pl_gt _metrics_cls/pred_weight_fg"]
# y_label = "Weights"
# # tag_names = ["FG ROI IoU wrt. pseudo-labels"]
# fname = f"correlation_ab1_default_{exp_name_ab1_default}.png"
# # threshs = np.array([[0.75, 0.75, 0.75], [0.65, 0.45, 0.4]])
# draw_metrics_correlation(dfs=dfs, fname=fname, tags=tags, y_label=y_label)